# 05. Plugin: Math Function
This tutorial shows you how to create plugins with Semantic Kernel. 
At the moment, Ollama with local LLMs doesn't support tool calling automatically. Hence we will have to use prompt to ask LLMs to return specified code and then parse the code to execute.

Beside this approach, you can also consider using llamasharp to call tools directly.

## Prerequisites

Refer to [01-get-started.ipynb](./01-get-started.ipynb) to install Ollama.

## Install Semantic Kernel

Run the following cell to install the packages.

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.11.1"

Installed Packages Microsoft.SemanticKernel, 1.11.1

## Instantiate the kernel

In [2]:
#pragma warning disable SKEXP0010
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

Action<string> println = System.Console.WriteLine;
Action<string> print = System.Console.Write;

// We use Mistral for this tutorial 
var modelId = "mistral";
// local Ollama endpoint
var endpoint = new Uri("http://localhost:11434");

var kernelBuilder = Kernel.CreateBuilder();
var kernel = kernelBuilder
    .AddOpenAIChatCompletion(
        modelId,
        endpoint,
        apiKey:null) 
    .Build();

#pragma warning restore SKEXP0010

##  Create the plugin

Let's first create a Math plugin with four functions `Add`, `Subtract`, `Multiply`, and `Divide`.


In [3]:
using System.ComponentModel;
using Microsoft.SemanticKernel;

public sealed class MyMathPlugin
{

    [KernelFunction, Description("Add two integers")]
    public static int Add(
        [Description("The first integer to add")] int number1,
        [Description("The second integer to add")] int number2
    )
    {
        return number1 + number2;
    }

    [KernelFunction, Description("Subtract two integers")]
    public static int Subtract(
        [Description("The first integer to subtract from")] int number1,
        [Description("The second integer to subtract away")] int number2
    )
    {
        return number1 - number2;
    }

    [KernelFunction, Description("Multiply two integers.")]
    public static int Multiply(
        [Description("The first integer to multiply")] int number1,
        [Description("The second integer to multiply")] int number2
    )
    {
        return number1 * number2;
    }

    [KernelFunction, Description("Divide two integers. Make sure the second integer is not 0.")]
    public static int Divide(
        [Description("The first integer to multiply")] int number1,
        [Description("The second integer to multiply")] int number2
    )
    {
        return number1 / number2;
    }
    
}

## Register the plugin with the kernel

After adding the plugin, we can then call it natively first.

In [4]:
public void PrintPlugins()
{
    foreach(var p in kernel.Plugins)
    println(p.Name);
}

In [5]:
if(kernel.Plugins.Count == 0)
{
    kernel.Plugins.AddFromType<MyMathPlugin>();
}

PrintPlugins();

KernelArguments args = new() {
    ["number1"]=10, 
    ["number2"]=20
};
var result = await kernel.InvokeAsync<int>("MyMathPlugin","Add", args);
Console.WriteLine(result);

MyMathPlugin
30


Let's explore the kernel function further. As you can see, Metadata property collects information of the native function which we can use in our prompt.

In [6]:
var addFunc = kernel.Plugins.GetFunction("MyMathPlugin", "Add");
addFunc

System.NotSupportedException: Serialization and deserialization of 'System.Type' instances is not supported. Path: $.Metadata.Parameters.ParameterType.\r\n ---> System.NotSupportedException: Serialization and deserialization of 'System.Type' instances is not supported.\r\n at System.Text.Json.Seri... Name Add PluginName MyMathPlugin Description Add two integers Metadata Microsoft.SemanticKernel.KernelFunctionMetadata Name Add PluginName MyMathPlugin Description Add two integers Parameters index value 0 Microsoft.SemanticKernel.KernelParameterMetadata Name number1 Description The first integer to add DefaultValue <null> IsRequired True ParameterType System.Int32 Schema {\r\n "type": "integer",\r\n "description": "The first integer to add"\r\n} RootElement {"type":"integer","description":"The first integer to add"} type integer description The first integer to add 1 Microsoft.SemanticKernel.KernelParameterMetadata Name number2 Description The second integer to add DefaultValue <null> IsRequired True ParameterType System.Int32 Schema {\r\n "type": "integer",\r\n "description": "The second integer to add"\r\n} RootElement {"type":"integer","description":"The second integer to add"} type integer description The second integer to add ReturnParameter Microsoft.SemanticKernel.KernelReturnParameterMetadata Description ParameterType System.Int32 Schema {\r\n "type": "integer"\r\n} RootElement {"type":"integer"} type "integer" AdditionalProperties (empty) ExecutionSettings <null>

In [7]:
var json = System.Text.Json.JsonSerializer.Serialize(
    kernel.Plugins.GetFunctionsMetadata().Select(f => 
        new{
            FunctionName = f.Name,
            f.PluginName,
            f.Description,
            Parameters = f.Parameters.Select(p => new {
                p.Name,
                p.Description
            })
        }
    )
    , new System.Text.Json.JsonSerializerOptions { WriteIndented = true }
);
display(json);

[
  {
    "FunctionName": "Add",
    "PluginName": "MyMathPlugin",
    "Description": "Add two integers",
    "Parameters": [
      {
        "Name": "number1",
        "Description": "The first integer to add"
      },
      {
        "Name": "number2",
        "Description": "The second integer to add"
      }
    ]
  },
  {
    "FunctionName": "Subtract",
    "PluginName": "MyMathPlugin",
    "Description": "Subtract two integers",
    "Parameters": [
      {
        "Name": "number1",
        "Description": "The first integer to subtract from"
      },
      {
        "Name": "number2",
        "Description": "The second integer to subtract away"
      }
    ]
  },
  {
    "FunctionName": "Multiply",
    "PluginName": "MyMathPlugin",
    "Description": "Multiply two integers.",
    "Parameters": [
      {
        "Name": "number1",
        "Description": "The first integer to multiply"
      },
      {
        "Name": "number2",
        "Description": "The second integer to multipl

## Try the plugin in Prompts

Now let's allow AI to response the function to invoke based on user input.

In [21]:
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;

var chatService = kernel.GetRequiredService<IChatCompletionService>();

var executionSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 4096,
    Temperature = 0,
    // Enable automatic invocation of kernel functions
    // Note this doesn't work with Ollama. It works with Azure OpenAI models. 
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var skPromptBuilder = new StringBuilder();
skPromptBuilder.AppendLine("""
## Introduction:

You are a conversational AI assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool in JSON format:

"""
);

skPromptBuilder.AppendLine("```json");
skPromptBuilder.AppendLine(json);
skPromptBuilder.AppendLine("```");

skPromptBuilder.AppendLine("""

## JSON Schema:

Here is the JSON format to response:

{
    "FunctionName": "Function name to answer the question",
    "PluginName": "The plugin name",
    "Parameters": [
      {
        "Name": "Parameter name, for example name1",
        "value": "The value for the parameter"
      },
    ],
    "Notes": "The reason why you chose this function."
}

## Task:

You need to answer the following question in the above JSON format. \
Answer EXACTLY AND ONLY in JSON format without any other comment or text. \
If you don't know the answer, return an empty JSON object.

Question:```{{$question}}```

""");

var skPrompt = skPromptBuilder.ToString();

var question = "What is 10 plus 20?";

// Let's print out the prompt
var promptTemplateConfig = new PromptTemplateConfig(skPrompt);
var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(promptTemplateConfig);
print(await promptTemplate.RenderAsync(kernel, new() { ["question"] = question}));


## Introduction:

You are a conversational AI assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool in JSON format:

```json
[
  {
    "FunctionName": "Add",
    "PluginName": "MyMathPlugin",
    "Description": "Add two integers",
    "Parameters": [
      {
        "Name": "number1",
        "Description": "The first integer to add"
      },
      {
        "Name": "number2",
        "Description": "The second integer to add"
      }
    ]
  },
  {
    "FunctionName": "Subtract",
    "PluginName": "MyMathPlugin",
    "Description": "Subtract two integers",
    "Parameters": [
      {
        "Name": "number1",
        "Description": "The first integer to subtract from"
      },
      {
        "Name": "number2",
        "Description": "The second integer to subtract away"
      }
    ]
  },
  {
    "FunctionName": "Multiply",
    "PluginName": "MyMathPlugin",
    "Description": "Multiply two integers.",
    "Parameters": [
      {


In [22]:
List<string> questions = [question, "What is 10 minus 20?", "What is 10 times 20?", "What is 10 divided by 20?"];

foreach(var q in questions)
{
    var response = await kernel.InvokePromptAsync<string>(skPrompt, new() { ["question"] = q });
    println($"Question: {q}");
    // response.Display();
    println($"Answer from the Model in JSON:");
    println(response);
}

Question: What is 10 plus 20?
Answer from the Model in JSON:
 {
  "FunctionName": "Add",
  "PluginName": "MyMathPlugin",
  "Parameters": [
    {
      "Name": "number1",
      "value": 10
    },
    {
      "Name": "number2",
      "value": 20
    }
  ],
  "Notes": "The question asks for 'What is 10 plus 20?', so the answer must be obtained by adding two numbers which can be achieved using the 'Add' function of 'MyMathPlugin'. Therefore, we use the PluginName: 'MyMathPlugin', FunctionName: 'Add' and provide the parameters number1: 10 and number2: 20."
}
Question: What is 10 minus 20?
Answer from the Model in JSON:
 {
  "FunctionName": "Subtract",
  "PluginName": "MyMathPlugin",
  "Parameters": [
    {
      "Name": "number1",
      "value": 10
    },
    {
      "Name": "number2",
      "value": 20
    }
  ],
  "Notes": "We are subtracting number2 (20) from number1 (10)"
}
Question: What is 10 times 20?
Answer from the Model in JSON:
 {
  "FunctionName": "Multiply",
  "PluginName": "My

## Call tool with the responsed JSON

Now we can use the responsed JSON to invoke the kernel function.

In [ ]:
// I will leave it to you to implement first using the above InvokeAsync method.